In [7]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [ ]:
#TGAC
file_deg = '../data/DEGs_by_RNAseq_21dpi_wheat.txt'
file_annotation = '/home/juan/Desktop/juan/bio/data/genomes/TGAC/Triticum_aestivum.TGACv1.39.clean.gff3'
dir_fasta = "/home/juan/Desktop/juan/bio/data/genomes/TGAC/by_scaff/"
file_out_fasta = "/home/juan/Desktop/juan/bio/mrcv/data/TGAC_DEG_SEQS.fasta"
gene_id = 'ID=gene:'
df_list = pd.read_csv(file_deg, index_col=False, sep='\t', comment='#')
genes = df_list.Gene.tolist() 
print(len(genes))

In [10]:
#IWGSC
file_deg = '/home/juan/Desktop/juan/bio/mrcv/data/res/rna_deg/DEG-normalized.csv'
file_annotation = '/home/juan/Desktop/juan/bio/mrcv/data/data/iwgsc/Triticum_aestivum.IWGSC.41.gff3'
dir_fasta = "/home/juan/Desktop/juan/bio/mrcv/data/data/iwgsc/by_chr/"
file_out_fasta = "/home/juan/Desktop/juan/bio/mrcv/data/res/DEG_SEQS.fasta"
gene_id = 'ID=gene:'
df_list = pd.read_csv(file_deg, index_col=False, sep=',', comment='#')
genes = df_list.gene.tolist() 
print(len(genes))

3019


In [11]:
df_gff = pd.read_csv(file_annotation, index_col=False, sep='\t', header=None, comment='#')
df_gff.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
df_gff = df_gff[df_gff.feature == 'gene']
print(len(df_gff.index))
df_gff.head(3)

107891


,seqname,source,feature,start,end,score,strand,frame,attribute
34,1A,IWGSC,gene,40098,70338,.,-,.,ID=gene:TraesCS1A02G000100;biotype=protein_cod...
65,1A,IWGSC,gene,70239,89245,.,+,.,ID=gene:TraesCS1A02G000200;biotype=protein_cod...
90,1A,IWGSC,gene,95906,104903,.,+,.,ID=gene:TraesCS1A02G000300;biotype=protein_cod...


In [ ]:
buffer_seqs = []
padding = 2000
for gene in genes:
    gene = gene.replace("gene:","")
    df_extract = df_gff[(df_gff.attribute.str.contains(gene_id + gene))]
    gene_gff = df_extract.iloc[0]
    end = max(gene_gff.end, gene_gff.start)
    start = min(gene_gff.end, gene_gff.start)
    start = max(start-padding, 0)
    end += padding
    record = SeqIO.read(dir_fasta + gene_gff.seqname + '.fasta', 'fasta')
    clean_seq = ''.join(str(record.seq).splitlines())
    new_seq = clean_seq[start:end]
    desc = "seq:" + str(gene_gff.seqname)
    desc += " start:" + str(start)
    desc += " end:" + str(end)
    desc += " strand:" + str(gene_gff.strand)
    seq = SeqRecord(Seq(new_seq), id=gene, description=desc)
    buffer_seqs.append(seq)


In [13]:
SeqIO.write(buffer_seqs, file_out_fasta, "fasta")

1143